<a href="https://colab.research.google.com/github/Inno732/Defi_IA_1_2023_2024/blob/main/Input_classification_Fire_detection_2023_2024_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Matériel : vérifier que le GPU est bien sélectionné (A COMPLETER)**

In [1]:
!nvidia-smi
!rm -rf sample_data

/bin/bash: line 1: nvidia-smi: command not found


# **2. Importation des librairies**

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2
import csv
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.applications.vgg16 import VGG16, preprocess_input #224*224
from keras.applications.xception import Xception
from tensorflow.keras.applications import ResNet50, DenseNet201
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
import math
from PIL import Image
from google.colab import files
print("Tensorflow version: "+tf.__version__)

from keras.applications.densenet import DenseNet121

from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from keras.callbacks import EarlyStopping

Tensorflow version: 2.14.0


#**3. Téléchargement des données d'entrainement "Small Dataset"**

In [3]:
bases_path_after="bases"
if os.path.exists(bases_path_after) == False:
    os.makedirs(bases_path_after)

In [4]:
!rm -rf sample_data
#! wget --no-check-certificate https://nextcloud.ig.umons.ac.be/s/QP7FoyttxJgS8Cj/download/small.zip
#! wget --no-check-certificate https://nextcloud.ig.umons.ac.be/s/REWbK6K4XRtoeNw/download/FIRE_DATABASE_1.zip
! wget --no-check-certificate https://nextcloud.ig.umons.ac.be/s/KaqzczZsXfsnMER/download/FIRE_DATABASE_2.zip # Base de données 3
#! wget --no-check-certificate https://nextcloud.ig.umons.ac.be/s/faKyDy7LCxfz9Xk/download/FIRE_DATABASE_3.zip # Base de données 2
#! unzip small.zip -d 'bases'
#! unzip FIRE_DATABASE_1.zip -d 'bases'
! unzip FIRE_DATABASE_2.zip -d 'bases'
#! unzip FIRE_DATABASE_3.zip -d 'bases'
#! rm small.zip
#! rm FIRE_DATABASE_1.zip
! rm FIRE_DATABASE_2.zip
#! rm FIRE_DATABASE_3.zip

--2023-11-24 11:40:07--  https://nextcloud.ig.umons.ac.be/s/KaqzczZsXfsnMER/download/FIRE_DATABASE_2.zip
Resolving nextcloud.ig.umons.ac.be (nextcloud.ig.umons.ac.be)... 193.190.209.220
Connecting to nextcloud.ig.umons.ac.be (nextcloud.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 466235525 (445M) [application/zip]
Saving to: ‘FIRE_DATABASE_2.zip’

FIRE_DATABASE_2.zip 100%[===================>] 444.64M  29.1MB/s    in 16s     

2023-11-24 11:40:25 (27.8 MB/s) - ‘FIRE_DATABASE_2.zip’ saved [466235525/466235525]

Archive:  FIRE_DATABASE_2.zip
   creating: bases/FIRE_DATABASE_3/
   creating: bases/FIRE_DATABASE_3/fire/
  inflating: bases/FIRE_DATABASE_3/fire/00000000.jpg  
  inflating: bases/FIRE_DATABASE_3/fire/00000001.jpg  
  inflating: bases/FIRE_DATABASE_3/fire/00000003.jpg  
  inflating: bases/FIRE_DATABASE_3/fire/00000004.jpg  
  inflating: bases/FIRE_DATABASE_3/fire/00000005.jpg  
  inflating: bases/FIRE_DATABASE_3/fire/0

#**4. Téléchargement des données de test "Test Dataset"**

In [5]:
!rm -rf sample_data
!wget https://nextcloud.ig.umons.ac.be/s/RsZpqtYfDgYzKxr/download/test.zip
!unzip test.zip -d 'bases'
!rm test.zip

--2023-11-24 11:40:37--  https://nextcloud.ig.umons.ac.be/s/RsZpqtYfDgYzKxr/download/test.zip
Resolving nextcloud.ig.umons.ac.be (nextcloud.ig.umons.ac.be)... 193.190.209.220
Connecting to nextcloud.ig.umons.ac.be (nextcloud.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57038454 (54M) [application/zip]
Saving to: ‘test.zip’

test.zip            100%[===================>]  54.40M  20.9MB/s    in 2.6s    

2023-11-24 11:40:41 (20.9 MB/s) - ‘test.zip’ saved [57038454/57038454]

Archive:  test.zip
   creating: bases/test/
   creating: bases/test/fire/
  inflating: bases/test/fire/F_1001.jpg  
  inflating: bases/test/fire/F_2062.jpg  
  inflating: bases/test/fire/F_2063.jpg  
  inflating: bases/test/fire/F_2064.jpg  
  inflating: bases/test/fire/F_2066.jpg  
  inflating: bases/test/fire/F_2067.jpg  
  inflating: bases/test/fire/F_2068.jpg  
  inflating: bases/test/fire/F_2070.jpg  
  inflating: bases/test/fire/F_2071.jpg  
  inflat

#**5. Création du fichier de labels "classes.txt"**

In [6]:
!printf '%s\n' 'fire' 'no_fire' 'start_fire'> classes.txt

#**6. Paramètres d'entrainement et sélection du modèle pré-entrainé**

In [7]:
nb_classes = 3
batch_size=8 #@param [1,2,4,8,16,32,64,128] {type:"raw"}
epochs=20 #@param [5, 10,20,50,100,200] {type:"raw"}
dataset_path = "bases"
input_dim=299 #@param [224,299] {type:"raw"}
dataset_name='FIRE_DATABASE_3' #@param ["small","FIRE_DATABASE_1","FIRE_DATABASE_2","FIRE_DATABASE_3"]
train_dataset = os.path.join('bases/', dataset_name)
test_name='test' #@param ["test","test1","test2","test3"]
test_dataset = os.path.join('bases/', test_name)
classes_path = "classes.txt"
classifier = "Xception" #@param ["Xception","VGG16","VGG19","ResNet50","ResNet121","ResNet152","ResNet50V2","ResNet121V2","ResNet152V2","InceptionV3","InceptionResNetV2","MobileNet","MobileNetV2","DenseNet121","DenseNet169","DenseNet201","NASNetLarge","NASNetMobile"] {type:"string"}


# **7. Compter le nombre de classes**

In [8]:
# Récupérer les noms des classes
with open(classes_path, 'r') as f:
    classes = f.readlines()
    classes = list(map(lambda x: x.strip(), classes))
num_classes = len(classes)

# **8. Création du jeu de données d'entraînement  (A COMPLETER)**

In [9]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
	train_dataset,               # chemin vers le jeu de données
	validation_split=0.2,        # paritionnement du jeu de données en un jeu de validation (20%), train (80%)
	subset="training",           # Sélection du jeu de données d'entraînement
	seed=42,                     # Initialisation du générateur aléatoire (permutations)
	image_size= (299, 299),      # Taille des images d'entrée
	batch_size= batch_size,      # Taille du mini-batch
  label_mode= 'categorical'    # Conversion au format One-Hot
)                             # Generates a 'tf.data.Dataset' from image files in a directory (train_dataset)
print(train_ds.class_names)

Found 1500 files belonging to 3 classes.
Using 1200 files for training.
['fire', 'no_fire', 'start_fire']


In [10]:
# class_names = train_ds.class_names

# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

#**9. Créer le jeu de validation (A COMPLETER : même générateur, One-Hot, jeu "validation")**

In [11]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dataset,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(299, 299),
    batch_size=batch_size,
    label_mode="categorical"
)
print(val_ds.class_names)

Found 1500 files belonging to 3 classes.
Using 300 files for validation.
['fire', 'no_fire', 'start_fire']


# **10. Télécharger un modèle préentraîné et appliquer le Transfer Learning**



In [12]:
callback = tf.keras.callbacks.EarlyStopping(monitor= 'val_loss', patience=3)  #  loss

In [13]:
model_VGG16 = VGG16(include_top = False, weights='imagenet', input_shape=(299, 299, 3))

# create a custom top classifier
x = model_VGG16.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1072, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model_VGG16 = Model(inputs = model_VGG16.inputs, outputs=predictions)

58889256/58889256 [==============================] - 0s 0us/step


In [14]:
# DenseNet121 model
DsNt121 = DenseNet121(weights= 'imagenet', include_top=False, input_shape=(299, 299,3))

# create a custom top classifier
x = DsNt121.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1072, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model_DsNt121 = Model(inputs = DsNt121.inputs, outputs=predictions)

29084464/29084464 [==============================] - 0s 0us/step


In [15]:
model_xception = Xception(include_top = False, weights='imagenet', input_shape=(299, 299, 3))

# create a custom top classifier
x = model_xception.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model_xception = Model(inputs = model_xception.inputs, outputs=predictions)


83683744/83683744 [==============================] - 1s 0us/step


In [16]:
model_VGG19 = VGG19(include_top = False, weights='imagenet', input_shape=(299, 299, 3))

# Freezing Layers
for layer in model_VGG19.layers:
    layer.trainable=False

# create a custom top classifier
x = model_VGG19.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1072, activation='relu')(x)
x = Dropout(0.2)(x)
#x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model_VGG19 = Model(inputs = model_VGG19.inputs, outputs=predictions)

80134624/80134624 [==============================] - 0s 0us/step


In [17]:
# ResNet50 model
resnet50 = ResNet50(include_top=False, weights='imagenet', input_shape=(299, 299,3))

for layer in resnet50.layers:
    layer.trainable = False

# build the entire model
x = resnet50.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten(name="flatten")(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.2)(x)

x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
#x = layers.Dense(256, activation='relu')(x)
#x = layers.Dropout(0.5)(x)
#x = layers.Dense(128, activation='relu')(x)
#x = layers.Dropout(0.5)(x)
#x = layers.Dense(64, activation='relu')(x)
#x = layers.Dropout(0.5)(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)
model_rsnt50 = Model(inputs = resnet50.input, outputs = predictions)

94765736/94765736 [==============================] - 1s 0us/step


In [18]:
from keras.applications import DenseNet121

# DenseNet201 model
Dsnt201 = DenseNet201(include_top=False, weights='imagenet', input_shape=(299, 299,3))

for layer in Dsnt201.layers:
    layer.trainable = False

# build the entire model
x = Dsnt201.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten(name="flatten")(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1072, activation='relu')(x)

predictions = layers.Dense(num_classes, activation='softmax')(x)
model_dsnt201 = Model(inputs = Dsnt201.input, outputs = predictions)

74836368/74836368 [==============================] - 0s 0us/step


# **10.1 Analyser/visualiser l'architecture de votre modèle avec la fonction summary()**

# **11. Geler puis dégeler les poids des premières couches**

In [19]:
for layer in model_xception.layers:
        layer.trainable = True

In [20]:
for layer in model_VGG16.layers:
        layer.trainable = True

# **11. Compiler le modèle avec la descente de gradient stochastique DSG**


In [21]:
# recompiler le modèle
model_xception.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [22]:
model_VGG16.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [23]:
model_rsnt50.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [24]:
model_DsNt121.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [25]:
model_dsnt201.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [26]:
model_VGG19.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [27]:
# model_xception.summary()

In [28]:
# model_VGG16.summary()

In [29]:
# model_rsnt50.summary()

# **12. Entraîner le modèle**

In [ ]:
history_VGG16 = model_VGG16.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    #callbacks=[callback],verbose=1
)

Epoch 1/20
  2/150 [..............................] - ETA: 1:18:56 - loss: 1.1461 - categorical_accuracy: 0.3750

In [ ]:
history_DsNt121 = model_DsNt121.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

In [ ]:
history_xception=model_xception.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

In [ ]:
history_VGG19 = model_VGG19.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

In [ ]:
history_dsnt201 = model_dsnt201.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

In [ ]:
history_rsnt50 = model_rsnt50.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

#**13. Sauvegarder le modèle final (A COMPLETER)**


In [ ]:
model_VGG16.save("model_VGG16.h5")

In [ ]:
model_DsNt121.save("model_DsNt121.h5")

In [ ]:
model_xception.save("model_xception.h5")

In [ ]:
model_rsnt50.save("model_rsnt50.h5")

In [ ]:
model_dsnt201.save("model_dsnt201.h5")

In [ ]:
model_VGG19.save("model_VGG19.h5")

#**14. Afficher des courbes d'entrainement/validation**

In [ ]:
fig, axs= plt.subplots(nrows=1, ncols=2, figsize=(20,5))
fig.suptitle('model score')

axs[0].plot(history_xception.history["categorical_accuracy"])
axs[0].plot(history_xception.history["val_categorical_accuracy"])
axs[0].plot(history_VGG16.history["categorical_accuracy"])
axs[0].plot(history_VGG16.history["val_categorical_accuracy"])
axs[0].plot(history_rsnt50.history["categorical_accuracy"])
axs[0].plot(history_rsnt50.history["val_categorical_accuracy"])
axs[0].plot(history_dsnt201.history["categorical_accuracy"])
axs[0].plot(history_dsnt201.history["val_categorical_accuracy"])
axs[0].plot(history_DsNt121.history["categorical_accuracy"])
axs[0].plot(history_DsNt121.history["val_categorical_accuracy"])
axs[0].plot(history_VGG19.history["categorical_accuracy"])
axs[0].plot(history_VGG19.history["val_categorical_accuracy"])

axs[0].legend(['X_train', 'X_vali', 'V16_train', 'V16_vali', 'R_train', 'R_vali', 'd201_train', 'd201_vali', 'd121_train', 'd121_vali', 'V19_train', 'V19_vali'], loc='upper left')
axs[0].set_title('Model accuracy')
axs[0].set_xlabel("epochs")
axs[0].set_ylabel("accuracy")

axs[1].plot(history_xception.history["loss"])
axs[1].plot(history_xception.history["val_loss"])
axs[1].plot(history_VGG16.history["loss"])
axs[1].plot(history_VGG16.history["val_loss"])
axs[1].plot(history_rsnt50.history["loss"])
axs[1].plot(history_rsnt50.history["val_loss"])
axs[1].plot(history_dsnt201.history["loss"])
axs[1].plot(history_dsnt201.history["val_loss"])
axs[1].plot(history_DsNt121.history["loss"])
axs[1].plot(history_DsNt121.history["val_loss"])
axs[1].plot(history_VGG19.history["loss"])
axs[1].plot(history_VGG19.history["val_loss"])
axs[1].legend(['X_train', 'X_vali', 'V_train', 'V_vali', 'R_train', 'R_vali', 'd201_train', 'd201_vali', 'd121_train', 'd121_vali', 'V19_train', 'V19_vali'], loc='upper left')
axs[1].set_title('Model loss')
axs[1].set_xlabel("epochs")
axs[1].set_ylabel("loss")

**Présenter votre meilleure courbe d’entrainement**

In [ ]:
fig, axs= plt.subplots(nrows=1, ncols=2, figsize=(20,5))
fig.suptitle('model VGG16 Score')

axs[0].plot(history_VGG16.history["categorical_accuracy"])
axs[0].plot(history_VGG16.history["val_categorical_accuracy"])

axs[0].legend(['VGG16_train', 'VGG16_vali'], loc='upper left')
axs[0].set_title('VGG16 accuracy')
axs[0].set_xlabel("epochs")
axs[0].set_ylabel("accuracy")

axs[1].plot(history_DsNt121.history["loss"])
axs[1].plot(history_DsNt121.history["val_loss"])
axs[1].legend(['VGG16_train', 'VGG16_vali'], loc='upper left')
axs[1].set_title('VGG16 loss')
axs[1].set_xlabel("epochs")
axs[1].set_ylabel("loss")

# **15. Créer le jeu de données de test**

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dataset,
    seed=42,
    image_size=(299, 299),
    batch_size=batch_size,
    label_mode="categorical"
)

In [ ]:
"""for i in range(len(val_preds)):
  if val_preds[i] == 0:
     val_preds[i] = 'fire'
  elif val_preds[i] == 1:
    val_preds[i] = 'no_fire'
  else:
    val_preds[i] = 'start_fire'
val_preds"""

In [ ]:
"""Y_pred = model_DsNt121.predict(test_ds, test_ds)
val_preds = np.argmax(Y_pred, axis=1)
test_ds.class_names"""

# **16. Evaluer le modèle avec ce jeu de données de test avec la fonction evaluate()**

In [ ]:
test = model_VGG16.evaluate(test_ds)

In [ ]:
test = model_DsNt121.evaluate(test_ds)

In [ ]:
test = model_xception.evaluate(test_ds)

In [ ]:
test = model_rsnt50.evaluate(test_ds)

In [ ]:
test = model_VGG19.evaluate(test_ds)

In [ ]:
test = model_dsnt201.evaluate(test_ds)

#**16. Tester le modèle avec une image de test: "fire.jpg" et "no_fire.jpg (A COMPLETER)"**

In [ ]:
files.upload()

In [ ]:
%matplotlib inline

axes=[]
fig=plt.figure()
fig = plt.figure(figsize=(18,9))

classes = train_ds.class_names
image_path1 =  "fire.jpg"
image_path2 = "no_fire.jpg"
image_path3 = "start_fire.jpg"
liste_image = [image_path1, image_path2, image_path3]

compteur = 1
for image_path in liste_image:
    # predict Xception.
    img = Image.open(image_path).convert('RGB')
    x = tf.keras.utils.img_to_array(img,data_format='channels_last')
    x = tf.keras.preprocessing.image.smart_resize(x, size=(299,299))
    x = np.expand_dims(x, axis=0)
    pred1 = model_VGG16.predict(x,batch_size=1)[0]

    # predict VGG16 et VGG19.
    img = Image.open(image_path).convert('RGB')
    x = tf.keras.utils.img_to_array(img,data_format='channels_last')
    x = tf.keras.preprocessing.image.smart_resize(x, size=(229,229))
    x = np.expand_dims(x, axis=0)
    pred2 = model_DsNt121.predict(x,batch_size=1)[0]
    pred3 = model_xception.predict(x,batch_size=1)[0]

    liste = [pred1, pred2, pred3]
    for pred in liste:
        for (pos,prob) in enumerate(pred):
            class_name = classes[pos]
            if (pos == np.argmax(pred)) :
                img = cv2.imread(image_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                font = cv2.FONT_HERSHEY_COMPLEX
                textsize = cv2.getTextSize(class_name, font, 1, 2)[0]
                textX = (img.shape[1] - textsize[0]) / 2
                textY = (img.shape[0] + textsize[1]) / 2
                cv2.putText(img, class_name, (int(textX)-10, int(textY)), font, 2, (255,0,0), 6, cv2.LINE_AA)
                axes.append(fig.add_subplot(3, 3, compteur))
                plt.imshow(img)
                compteur += 1
            #print("Top %d ====================" % (i + 1))
            print("Class Name : %s" % (class_name), "---", "Class Probability: %.2f%%" % (prob*100))
        print("----------------------")

plt.show()

In [ ]:
%matplotlib inline

classes = train_ds.class_names
image_path =  "no_fire.jpg"

img = Image.open(image_path).convert('RGB')
x = tf.keras.utils.img_to_array(img,data_format='channels_last')
x = tf.keras.preprocessing.image.smart_resize(x, size=(input_dim,input_dim))
x = np.expand_dims(x, axis=0)

# predict
#pred = model_xception.predict(x,batch_size=1)[0]
pred = model_DsNt121.predict(x,batch_size=1)[0]

for (pos,prob) in enumerate(pred):
    class_name = classes[pos]
    if (pos == np.argmax(pred)) :
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        font = cv2.FONT_HERSHEY_COMPLEX
        textsize = cv2.getTextSize(class_name, font, 1, 2)[0]
        textX = (img.shape[1] - textsize[0]) / 2
        textY = (img.shape[0] + textsize[1]) / 2
        cv2.putText(img, class_name, (int(textX)-10, int(textY)), font, 2, (255,0,0), 6, cv2.LINE_AA)
        plt.imshow(img)
    #print("Top %d ====================" % (i + 1))
    print("Class Name : %s" % (class_name), "---", "Class Probability: %.2f%%" % (prob*100))
plt.show()

In [ ]:
classes = train_ds.class_names
image_path =  "no_fire.jpg"

img = Image.open(image_path).convert('RGB')
x = tf.keras.utils.img_to_array(img,data_format='channels_last')
x = tf.keras.preprocessing.image.smart_resize(x, size=(input_dim,input_dim))
x = np.expand_dims(x, axis=0)

# predict
pred = model_VGG16.predict(x,batch_size=1)[0]

for (pos,prob) in enumerate(pred):
    class_name = classes[pos]
    if (pos == np.argmax(pred)) :
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        font = cv2.FONT_HERSHEY_COMPLEX
        textsize = cv2.getTextSize(class_name, font, 1, 2)[0]
        textX = (img.shape[1] - textsize[0]) / 2
        textY = (img.shape[0] + textsize[1]) / 2
        cv2.putText(img, class_name, (int(textX)-10, int(textY)), font, 2, (255,0,0), 6, cv2.LINE_AA)
        plt.imshow(img)
    #print("Top %d ====================" % (i + 1))
    print("Class Name : %s" % (class_name), "---", "Class Probability: %.2f%%" % (prob*100))
plt.show()

In [ ]:
# créer les jeux de données d'ntrainement 90%
train_ds_0 = tf.keras.preprocessing.image_dataset_from_directory(
	train_dataset,               # chemin vers le jeu de données
	validation_split=0.1,        # paritionnement du jeu de données en un jeu de validation (20%), train (80%)
	subset="training",           # Sélection du jeu de données d'entraînement
	seed=42,                     # Initialisation du générateur aléatoire (permutations)
	image_size= (299, 299),      # Taille des images d'entrée
	batch_size= batch_size,      # Taille du mini-batch
  label_mode= 'categorical'    # Conversion au format One-Hot
)

# données de validation 10%
val_ds_0 = tf.keras.preprocessing.image_dataset_from_directory(
    train_dataset,
    validation_split=0.1,
    subset="validation",
    seed=42,
    image_size=(299, 299),
    batch_size=batch_size,
    label_mode="categorical"
)

# créer les jeux de données d'ntrainement 80%
train_ds_1 = tf.keras.preprocessing.image_dataset_from_directory(
	train_dataset,               # chemin vers le jeu de données
	validation_split=0.2,        # paritionnement du jeu de données en un jeu de validation (20%), train (80%)
	subset="training",           # Sélection du jeu de données d'entraînement
	seed=42,                     # Initialisation du générateur aléatoire (permutations)
	image_size= (299, 299),      # Taille des images d'entrée
	batch_size= batch_size,      # Taille du mini-batch
  label_mode= 'categorical'    # Conversion au format One-Hot
)

# données de validation 20%
val_ds_1 = tf.keras.preprocessing.image_dataset_from_directory(
    train_dataset,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(299, 299),
    batch_size=batch_size,
    label_mode="categorical"
)

# créer les jeux de données d'ntrainement 50%
train_ds_2 = tf.keras.preprocessing.image_dataset_from_directory(
	train_dataset,               # chemin vers le jeu de données
	validation_split=0.5,        # paritionnement du jeu de données en un jeu de validation (20%), train (80%)
	subset="training",           # Sélection du jeu de données d'entraînement
	seed=42,                     # Initialisation du générateur aléatoire (permutations)
	image_size= (299, 299),      # Taille des images d'entrée
	batch_size= batch_size,      # Taille du mini-batch
  label_mode= 'categorical'    # Conversion au format One-Hot
)

# données de validation 50%
val_ds_2 = tf.keras.preprocessing.image_dataset_from_directory(
    train_dataset,
    validation_split=0.5,
    subset="validation",
    seed=42,
    image_size=(299, 299),
    batch_size=batch_size,
    label_mode="categorical"
)

# créer les jeux de données d'ntrainement 20%
train_ds_3 = tf.keras.preprocessing.image_dataset_from_directory(
	train_dataset,               # chemin vers le jeu de données
	validation_split=0.8,        # paritionnement du jeu de données en un jeu de validation (20%), train (80%)
	subset="training",           # Sélection du jeu de données d'entraînement
	seed=42,                     # Initialisation du générateur aléatoire (permutations)
	image_size= (299, 299),      # Taille des images d'entrée
	batch_size= batch_size,      # Taille du mini-batch
  label_mode= 'categorical'    # Conversion au format One-Hot
)

# données de validation 80%
val_ds_3 = tf.keras.preprocessing.image_dataset_from_directory(
    train_dataset,
    validation_split=0.8,
    subset="validation",
    seed=42,
    image_size=(299, 299),
    batch_size=batch_size,
    label_mode="categorical"
)

In [ ]:
# DenseNet121 model
DsNt121 = DenseNet121(weights= 'imagenet', include_top=False, input_shape=(299, 299,3))
# create a custom top classifier
x = DsNt121.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1072, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model0_DsNt121 = Model(inputs = DsNt121.inputs, outputs=predictions)

DsNt121 = DenseNet121(weights= 'imagenet', include_top=False, input_shape=(299, 299,3))
# create a custom top classifier
x = DsNt121.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1072, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model1_DsNt121 = Model(inputs = DsNt121.inputs, outputs=predictions)

DsNt121 = DenseNet121(weights= 'imagenet', include_top=False, input_shape=(299, 299,3))
# create a custom top classifier
x = DsNt121.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1072, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model2_DsNt121 = Model(inputs = DsNt121.inputs, outputs=predictions)

DsNt121 = DenseNet121(weights= 'imagenet', include_top=False, input_shape=(299, 299,3))
# create a custom top classifier
x = DsNt121.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1072, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model3_DsNt121 = Model(inputs = DsNt121.inputs, outputs=predictions)



In [ ]:
   # compiler le modèle
model0_DsNt121.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

model1_DsNt121.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

model2_DsNt121.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

model3_DsNt121.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# entrainer le modèle
history_0 = model0_DsNt121.fit(
    train_ds_0,
    epochs=epochs,
    validation_data=val_ds_0
)

In [ ]:
# entrainer le modèle
history_1 = model1_DsNt121.fit(
    train_ds_1,
    epochs=epochs,
    validation_data=val_ds_1
)

In [ ]:
history_2 = model2_DsNt121.fit(
    train_ds_2,
    epochs=epochs,
    validation_data=val_ds_2
)

In [ ]:
history_3 = model3_DsNt121.fit(
    train_ds_3,
    epochs=epochs,
    validation_data=val_ds_3
)

In [ ]:
fig, axs= plt.subplots(nrows=2, ncols=4, figsize=(20,5))
#fig.suptitle('Rates')

axs[0, 0].plot(history_0.history["categorical_accuracy"])
axs[0, 0].plot(history_0.history["val_categorical_accuracy"])
axs[0, 0].legend(['Train', 'Val'], loc='upper left')
axs[0, 0].set_title('90% Train and 10% Validation')
#axs[0, 0].set_xlabel("epochs")
axs[0, 0].set_ylabel("accuracy")

axs[1, 0].plot(history_0.history["loss"])
axs[1, 0].plot(history_0.history["val_loss"])
axs[1, 0].legend(['Train', 'Vali'], loc='upper left')
#axs[1, 0].set_title('90% Train and 10% Validation')
axs[1, 0].set_xlabel("epochs")
axs[1, 0].set_ylabel("loss")

axs[0, 1].plot(history_1.history["categorical_accuracy"])
axs[0, 1].plot(history_1.history["val_categorical_accuracy"])
axs[0, 1].legend(['Train', 'Val'], loc='upper left')
axs[0, 1].set_title('80% Train and 20% Validation')
#axs[0, 1].set_xlabel("epochs")
#axs[0, 1].set_ylabel("accuracy")

axs[1, 1].plot(history_1.history["loss"])
axs[1, 1].plot(history_1.history["val_loss"])
axs[1, 1].legend(['Train', 'Vali'], loc='upper left')
#axs[1, 1].set_title('80% Train and 20% Validation')
axs[1, 1].set_xlabel("epochs")
#axs[1, 1].set_ylabel("loss")

axs[0, 2].plot(history_2.history["categorical_accuracy"])
axs[0, 2].plot(history_2.history["val_categorical_accuracy"])
axs[0, 2].legend(['Train', 'Val'], loc='upper left')
axs[0, 2].set_title('50% Train and 50% Validation')
#axs[0, 2].set_xlabel("epochs")
#axs[0, 2].set_ylabel("accuracy")

axs[1, 2].plot(history_2.history["loss"])
axs[1, 2].plot(history_2.history["val_loss"])
axs[1, 2].legend(['Train', 'Vali'], loc='upper left')
#axs[1, 2].set_title('50% Train and 50% Validation')
axs[1, 2].set_xlabel("epochs")
#axs[1, 2].set_ylabel("loss")

axs[0, 3].plot(history_3.history["categorical_accuracy"])
axs[0, 3].plot(history_3.history["val_categorical_accuracy"])
axs[0, 3].legend(['Train', 'Val'], loc='upper left')
axs[0, 3].set_title('20% Train and 80% Validation')
#axs[0, 3].set_xlabel("epochs")
#axs[0, 3].set_ylabel("accuracy")

axs[1, 3].plot(history_3.history["loss"])
axs[1, 3].plot(history_3.history["val_loss"])
axs[1, 3].legend(['Train', 'Vali'], loc='upper left')
#axs[1, 3].set_title('20% Train and 80% Validation')
axs[1, 3].set_xlabel("epochs")
#axs[1, 3].set_ylabel("loss")

In [ ]:
# Evaluation des modèles sur le test
test_0 = model0_DsNt121.evaluate(test_ds)

In [ ]:
test_1 = model1_DsNt121.evaluate(test_ds)

In [ ]:
test_2 = model2_DsNt121.evaluate(test_ds)

In [ ]:
test_3 = model3_DsNt121.evaluate(test_ds)

In [ ]:
model0_DsNt121.save("model0_DsNt121.h5")

In [ ]:
model1_DsNt121.save("model1_DsNt121.h5")

In [ ]:
model2_DsNt121.save("model2_DsNt121.h5")

In [ ]:
model3_DsNt121.save("model3_DsNt121.h5")